In [1]:
from ase.atoms import Atoms
from ase.io import write
from ase.visualize import view
import e3nn_jax as e3nn
import jax
import jax.numpy as jnp
import jraph
import plotly.express as px
import plotly.graph_objects as go
import sys
import tqdm

sys.path.append('..')
sys.path.append('../analyses')
import analysis
import datatypes
import input_pipeline_tf
import models
import train

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
atomic_numbers = jnp.array([1, 6, 7, 8, 9])
numbers_to_symbols = {1: 'H', 6: 'C', 7: 'N', 8: 'O', 9: 'F'}
elements = list(numbers_to_symbols.values())

# covalent bond radii, in angstroms
element_radii = [0.32, 0.75, 0.71, 0.63, 0.64]

def get_numbers(species: jnp.ndarray):
    numbers = []
    for i in species:
        numbers.append(atomic_numbers[i])
    return jnp.array(numbers)

In [3]:
config, best_state_train, best_state_eval, metrics_for_best_state = analysis.load_from_workdir('../workdirs/mace/interactions=2/l=3/channels=32', load_pickled_params=False)
rng = jax.random.PRNGKey(config.rng_seed)
rng, dataset_rng = jax.random.split(rng)
datasets = input_pipeline_tf.get_datasets(dataset_rng, config)

cutoff = 5.0
epsilon = 1e-4

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [4]:
losses = []

for i in tqdm.tqdm(range(1000)):
    graph = next(datasets["test"].as_numpy_iterator())
    frag = datatypes.Fragment.from_graphstuple(graph)
    frag = jax.tree_map(jnp.asarray, frag)

    frag_unpadded = jraph.unpad_with_graphs(frag)
    molecules = jraph.unbatch(frag_unpadded)

    for j, mol in enumerate(molecules):
        preds = train.get_predictions(best_state_train, mol, rng)
        mol_loss = train.generation_loss(preds, mol, config.loss_kwargs.radius_rbf_variance)
        losses.append((mol_loss, i, j))

  0%|          | 0/1000 [02:28<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
losses = sorted(losses)